Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [369]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

ENABLE_PREPROCESSING=False

In [370]:
State = namedtuple('State', ['taken', 'not_taken'])

def join_taken(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(join_taken(state))

In [371]:
PROBLEM_SIZE = 10
NUM_SETS = 10
SETS = list(np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"


In [372]:
# preproccesing
def forced_sets():
    check_covered_by_one=np.vstack(SETS).sum(0)

    if 1 not in check_covered_by_one:
        return State(set(),set(range(NUM_SETS)))

    forced_list = [
        index
        for index, s in enumerate(SETS)
        if any(check_covered_by_one[i] == 1 and s[i] == True for i in range(PROBLEM_SIZE))
    ]
    return State(set(forced_list), set(range(NUM_SETS)) - set(forced_list))

def set_sorting_preprocessing():
    return sorted(SETS,reverse=True,key=lambda e: sum(e)) #ordering respect ascending number of True is better
    

In [373]:

def missing_coverage(state):
    return PROBLEM_SIZE-sum(state)

def calc_overlap_and_false_number(state,e):
    return sum( np.logical_and(
        state, SETS[e])) + missing_coverage(SETS[e])

def calc_extension(state,e):
    ext=0
    for i in state.taken:
        for j in range(PROBLEM_SIZE):
            if SETS[i][j]==False and SETS[e][j]==True:
                ext+=1
    return ext

def my_h(state,e):    
    return calc_overlap_and_false_number(join_taken(state), e)+calc_extension(state,e)

def my_h2(state):
    joined_taken=join_taken(state)
    
    not_taken_sorted_by_overlap=sorted(state.not_taken,key=lambda e:calc_overlap_and_false_number(joined_taken,e))    

    sets_still_needed=0
    for s in not_taken_sorted_by_overlap:
        
        if missing_coverage(joined_taken)==0:
            break
        joined_taken = np.logical_or(joined_taken,SETS[s])
        sets_still_needed+=1
        
    return sets_still_needed    

def f(state):
    #return len(state[0].taken)+my_h(state[0],state[1])
    return len(state.taken) + my_h2(state)

In [374]:
frontier = PriorityQueue()

if ENABLE_PREPROCESSING==True:
    state = forced_sets()
    SETS=set_sorting_preprocessing()
else:
    state = State(set(), set(range(NUM_SETS)))
    
frontier.put((0, state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        #cost=f((current_state,action))
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
            #frontier.put((cost, new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

7it [00:00, 701.61it/s]

Solved in 7 steps (3 tiles)


In [375]:
current_state.taken

{1, 8, 9}